In [1]:
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium import webdriver

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘


dart_list = dart.list(start='20241101',end=today)
lists = dart_list[dart_list['report_nm'].str.contains('기업설명회')]
print(f'{today}_{len(lists)}건')

def investment_briefing(rcp_no,corp):
    
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt.T
    if '장소' in tt.iloc[0,0]:
        tt = tt[[0,1,2,3,4,5,6,7,8,9]]
        tt.columns = ['일시','장소','참가대상','목적','방법','후원기관','요약','결정일','게재일시','웹페이지']
        tt = tt.drop(index=[0,1,3])
        tt = tt.dropna(axis=1)
        tt.insert(0,'기업',corp)
        tt.insert(0,'조회날짜',today)
    
    else:
        tt['시작일']=tt[2].shift(1)
        tt['일시'] = tt['시작일']+'~'+tt[2]
        tt = tt[['일시',3,4,5,6,8,7,9,12,13]]
        tt.columns = ['일시','장소','참가대상','목적','방법','후원기관','요약','결정일','게재일시','웹페이지']
        tt = tt.iloc[[2],:]
        tt.insert(0,'기업',corp)
        tt.insert(0,'조회날짜',today)
    return tt

df = pd.DataFrame()

for i in range(len(lists)):
    try:
        # rcp_no = '20240822900430'
        # corp = 'bgf'
        rcp_no = lists['rcept_no'].values[i]
        corp = lists['flr_nm'].values[i]
        ir_df = investment_briefing(rcp_no,corp)


        print(f'{i}_{corp} 완료')  
        df = pd.concat([df,ir_df])
    except Exception as e:
        print(f'{corp}',e)
        
file_name = f'C:\투자기록\정리\코딩\장내매수\기업설명회_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
df.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()

20241110_133건
20241108800871 이수페타시스
0_이수페타시스 완료
20241108600793 태양기계
태양기계 '[13] not in index'
20241108800772 CJ
CJ 2
20241108800678 시프트업
3_시프트업 완료
20241108800669 시프트업
4_시프트업 완료
20241108800627 SK디앤디
5_SK디앤디 완료
20241108800605 카카오
6_카카오 완료
20241108900593 현대바이오랜드
7_현대바이오랜드 완료
20241108900554 파크시스템스
8_파크시스템스 완료
20241108900548 펄어비스
9_펄어비스 완료
20241108800520 HL홀딩스
10_HL홀딩스 완료
20241108900479 코스메카코리아
11_코스메카코리아 완료
20241108900440 잉글우드랩
12_잉글우드랩 완료
20241108900454 CJ프레시웨이
13_CJ프레시웨이 완료
20241108900446 네오위즈
14_네오위즈 완료
20241108900432 엠로
15_엠로 완료
20241108800421 한국콜마
16_한국콜마 완료
20241108600391 관악산업
관악산업 '[13] not in index'
20241108800386 한미약품
18_한미약품 완료
20241108800350 미래에셋증권
19_미래에셋증권 완료
20241108900327 알테오젠
20_알테오젠 완료
20241108800321 한국전력공사
21_한국전력공사 완료
20241108800260 신세계
22_신세계 완료
20241108900256 이지케어텍
23_이지케어텍 완료
20241108800230 CJ대한통운
24_CJ대한통운 완료
20241108800187 현대지에프홀딩스
25_현대지에프홀딩스 완료
20241108800186 현대지에프홀딩스
26_현대지에프홀딩스 완료
20241108800183 현대지에프홀딩스
27_현대지에프홀딩스 완료
20241108900164 에이프릴바이오
28_에이프릴바이오 완료
2024110

In [53]:
file_name = f'C:\투자기록\정리\코딩\장내매수\기업설명회_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
df.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()
